## (A) 기존 SigLIP2 모델 사용

In [2]:
from transformers import pipeline

pipe = pipeline(
    model="google/siglip2-large-patch16-256",
    task="zero-shot-image-classification",
    device=-1
)

# 후보 라벨 목록 (텍스트로 된 클래스 이름들)
labels = ["흰색의 분말이 들어있는 상부 초록색, 하부 미황색의 경질캡슐제, 앞면 '마크 ELT', 뒷면 '' 라고 적혀 있음", 
          "백색의 원형필름제피정, 앞면 'SJΛ', 뒷면 ''라고 적혀 있음",
          "미황색의 원형 정제, 앞면 'YH', 뒷면 ''라고 적혀 있음",
          "백색의 원형 정제, 앞면 'YH', 뒷면 ''라고 적혀 있음",
          "흰색 또는 거의 흰색의 원형 정제, 앞면 '마크분할선ST', 뒷면 ''라고 적혀 있음",
          "장방형 백색 정제, 앞면 'LDI분할선LDI', 뒷면 '분할선'라고 적혀 있음"]  # 총 4000개 가능

# 알약 이미지에 대해 예측
result = pipe(r"C:\Users\302-09\Desktop\깡통\my알약\KakaoTalk_20250612_144853880_02.jpg", candidate_labels=labels)
print("🔍 예측 결과:")
for i, res in enumerate(result, 1):
    print(f"{i}. Label: {res['label']}")
    print(f"   Score: {res['score']:.4f}")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


🔍 예측 결과:
1. Label: 흰색의 분말이 들어있는 상부 초록색, 하부 미황색의 경질캡슐제, 앞면 '마크 ELT', 뒷면 '' 라고 적혀 있음
   Score: 0.2955
2. Label: 미황색의 원형 정제, 앞면 'YH', 뒷면 ''라고 적혀 있음
   Score: 0.0056
3. Label: 백색의 원형 정제, 앞면 'YH', 뒷면 ''라고 적혀 있음
   Score: 0.0008
4. Label: 백색의 원형필름제피정, 앞면 'SJΛ', 뒷면 ''라고 적혀 있음
   Score: 0.0008
5. Label: 흰색 또는 거의 흰색의 원형 정제, 앞면 '마크분할선ST', 뒷면 ''라고 적혀 있음
   Score: 0.0005
6. Label: 장방형 백색 정제, 앞면 'LDI분할선LDI', 뒷면 '분할선'라고 적혀 있음
   Score: 0.0003


## (B) 기존 SigLIP2 모델에 알약을 학습

X

## (C) 기존 SigLIP2 모델 + 임베딩, 코사인 유사도

In [1]:
from transformers import AutoProcessor, AutoModel
import torch
from PIL import Image

# 모델 준비 (사전 학습된 SigLIP2)
model_name = "google/siglip2-large-patch16-256"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

# 이미지 임베딩 추출 함수
def get_image_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        img_embed = model.get_image_features(**inputs)
    return img_embed / img_embed.norm(dim=-1, keepdim=True)

# 텍스트 임베딩 추출 함수
def get_text_embeddings(text_list):
    inputs = processor(text=text_list, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        text_embeds = model.get_text_features(**inputs)
    return text_embeds / text_embeds.norm(dim=-1, keepdim=True)

# 🔹 후보 텍스트 설명들 (예: seq 기반 설명들)
text_labels = ["흰색의 분말이 들어있는 상부 초록색, 하부 미황색의 경질캡슐제, 앞면 '마크 ELT', 뒷면 '' 라고 적혀 있음", 
          "백색의 원형필름제피정, 앞면 'SJΛ', 뒷면 ''라고 적혀 있음",
          "미황색의 원형 정제, 앞면 'YH', 뒷면 ''라고 적혀 있음",
          "백색의 원형 정제, 앞면 'YH', 뒷면 ''라고 적혀 있음",
          "흰색 또는 거의 흰색의 원형 정제, 앞면 '마크분할선ST', 뒷면 ''라고 적혀 있음",
          "장방형 백색 정제, 앞면 'LDI분할선LDI', 뒷면 '분할선'라고 적혀 있음"
          ]

# 텍스트 임베딩 (1회만)
text_embeds = get_text_embeddings(text_labels)

# 🔹 예측할 이미지
img_path = r"C:\Users\302-09\Desktop\깡통\my알약\KakaoTalk_20250612_144853880_02.jpg"
img_embed = get_image_embedding(img_path)

# 🔍 유사도 계산
similarities = (img_embed @ text_embeds.T).squeeze(0)
top_k = torch.topk(similarities, k=3)

# 결과 출력
print("🔍 예측 결과 (Top-3)")
for idx, score in zip(top_k.indices, top_k.values):
    print(f"- {text_labels[idx]} (score: {score:.4f})")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🔍 예측 결과 (Top-3)
- 백색의 원형 정제, 앞면 'YH', 뒷면 ''라고 적혀 있음 (score: 0.0886)
- 흰색 또는 거의 흰색의 원형 정제, 앞면 '마크분할선ST', 뒷면 ''라고 적혀 있음 (score: 0.0792)
- 미황색의 원형 정제, 앞면 'YH', 뒷면 ''라고 적혀 있음 (score: 0.0698)
